In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing, decomposition, model_selection
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput 


%matplotlib inline

In [2]:
data_dir = 'input_data'
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
data_prefix = 'santander_project/data'
seed=0
n_components=200

# Reading in training dataset
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'), index_col='ID')

## PCA Reduction
Here i choose to explore various kernels for a standard PCA approach, before ultimately selecting one for use. I also separately conducting a sparse PCA model using SageMaker. The computational cost of the sparse pca is such that i simply saved the output for potential use in modeling.

### Exploring Kernel-based PCA

First, comparing performance of various PCA kernels

In [35]:
# Scaling features to means
std_scale = preprocessing.StandardScaler().fit(train_df.iloc[:, 1:])
train_df_scaled = std_scale.transform(train_df.iloc[:, 1:])

# Applying Box-Cox transform to target
train_df['target'] = stats.boxcox(train_df['target'])[0]

In [ ]:
# Looping through kernels and plotting component correlations to target
kernels=['linear', 'poly', 'rbf', 'sigmoid', 'cosine']

fig = plt.figure(figsize=(20, 20))

kernel_outputs = {}

for i, kernel in enumerate(kernels):
    kpca_sm = decomposition.KernelPCA(n_components=n_components, kernel=kernel)
    kpca_sm.fit(train_df_scaled)
    train_df_kpca = pd.concat([train_df['target'], pd.DataFrame(kpca_sm.transform(train_df_scaled), columns=['c{}'.format(num+1) for num in range(n_components)], index=train_df.index)], axis=1)
    correlation_kpca = train_df_kpca.corr(method='spearman')['target'][1:]
    
    #Saving results in dictionary for post-evaluation use
    kernel_outputs[kernel] = {}
    kernel_outputs[kernel]['coeffs'] = correlation_kpca
    kernel_outputs[kernel]['data']= train_df_kpca
    kernel_outputs[kernel]['transformer'] = kpca_sm
    
    #Plotting results
    ax = fig.add_subplot(3, 2, i+1)
    ax.scatter(range(len(correlation_kpca)), np.sort(correlation_kpca))
    ax.set_title('Component/Target Correlation ({} Kernel)'.format(kernel))
    ax.set_xlabel('Components')
    ax.set_ylabel('Correlation')
    ax.set_ylim(-.3, .3)
    

The polynomial kernel appears to have the highest magnitude of correlations to the target. 

### Conducting Standard PCA Reduction and Saving Output

PCA using a polynomial model appeared to perform the best. Using a gridsearch to optimize the paramaters, assuming this will be ultimately fitted to a gradient boosting regression model. Selecting components meeting correlation threshold and saving to a dataframe for regression modeling.

In [4]:
# Optimizing parameters

poly_kpca = decomposition.KernelPCA(n_components=n_components, kernel='poly')
GBRegressor = GradientBoostingRegressor()

X = train_df_scaled
y = train_df['target'] 

pipe = Pipeline(steps=[('pca', poly_kpca), ('grb_regressor', GBRegressor)])

param_grid = {
    'pca__gamma': [1, 10, 100, 1000],
    'pca__degree': [3, 4, 5],
}

pca_search = GridSearchCV(pipe, param_grid, n_jobs=-1)
pca_search.fit(X, y)

pca_search.best_params_

{'pca__degree': 3, 'pca__gamma': 10}

In [5]:
optimized_kpca = pca_search.best_estimator_[0]

In [6]:
# Fitting PCA algorithm to data

optimized_kpca.fit(train_df_scaled)
train_df_optimized = pd.concat([train_df['target'], pd.DataFrame(optimized_kpca.transform(train_df_scaled), columns=['c{}'.format(num+1) for num in range(n_components)], index=train_df.index)], axis=1)
correlation_optimized = train_df_optimized.corr(method='spearman')['target'][1:]

In [7]:
# Reducing components based on correlation threshold

pca_threshold = .05
best_components = correlation_optimized[abs(correlation_optimized) >= pca_threshold].sort_values(ascending=False)
pca_cols = [['target'] + list(best_components.index)][0]
train_df_final_pca = train_df_optimized.loc[:, pca_cols]

In [9]:
#Saving dataframe of best components to local drive
train_df_final_pca.to_csv(os.path.join(data_dir, 'train_pca.csv'), header=False, index=False)

Transforming test data using fitted standard scaler and PCA model

In [36]:
#Transforming and saving test data using selected PCA kernel
chunksize = 5000 
for chunk in pd.read_csv(os.path.join(data_dir, 'test.csv'), index_col='ID', chunksize=chunksize):
        scaled_chunk = std_scale.transform(chunk.values)
        transformed_chunk = optimized_kpca.transform(scaled_chunk)
        transformed_chunk_df = pd.DataFrame(transformed_chunk, columns=['c{}'.format(num+1) for num in range(n_components)], index=chunk.index).loc[:, list(best_components.index)]
        transformed_chunk_df.to_csv(os.path.join(data_dir, 'test_pca.csv'), header=False, index=False, mode='a')

In [37]:
# Uploading transformed files to sagemaker for later use
sagemaker_session.upload_data(path=os.path.join(data_dir, 'train_pca.csv'), bucket=bucket, key_prefix=data_prefix)
sagemaker_session.upload_data(path=os.path.join(data_dir, 'test_pca.csv'), bucket=bucket, key_prefix=data_prefix)

's3://sagemaker-us-east-2-278383315865/santander_project/data/test_pca.csv'

### Conducting Sparse PCA Reduction and Saving Output

Evaluating the performance of sparse PCA on the dataset. Because of the computational cost, i am executing the transformation using the SageMaker API and saving the output for future use.

In [ ]:
#Getting data from S3
train_path = 's3://{}/{}/train.csv'.format(bucket, data_prefix)
test_path = 's3://{}/{}/test.csv'.format(bucket, data_prefix)

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.c4.xlarge',
                                     instance_count=1)

#Run imputation processing script
sklearn_processor.run(code='processing_scripts/sparse_PCA.py',
                      inputs=[ProcessingInput(
                        source=train_path,
                        destination='/opt/ml/processing/input/train'), ProcessingInput(
                        source=test_path,
                        destination='/opt/ml/processing/input/test'),],
                      outputs=[ProcessingOutput(output_name='train_pca',
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test_pca',
                                                source='/opt/ml/processing/test')]
                     )


In [ ]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

In [ ]:
preprocessing_job_description

In [ ]:
output_config = preprocessing_job_description['ProcessingOutputConfig']

In [ ]:
for output in output_config['Outputs']:
    if output['OutputName'] == 'train_pca':
        preprocessed_training_data = output['S3Output']['S3Uri']
    if output['OutputName'] == 'test_pca':
        preprocessed_test_data = output['S3Output']['S3Uri']


In [ ]:
pd.read_csv(preprocessed_training_data + '/train_sparse_pca.csv', index_col='ID').to_csv(os.path.join(data_dir, 'train_sparse_pca.csv'), header=False, index=False)
pd.read_csv(preprocessed_test_data + '/test_sparse_pca.csv', index_col='ID').to_csv(os.path.join(data_dir, 'test_sparse_pca.csv'), header=False, index=False)

In [ ]:
sagemaker_session.upload_data(path=os.path.join(data_dir, 'train_sparse_pca.csv'), bucket=bucket, key_prefix=data_prefix)
sagemaker_session.upload_data(path=os.path.join(data_dir, 'test_sparse_pca.csv'), bucket=bucket, key_prefix=data_prefix)

In [ ]:
correlation_sparse = train_df_sparse.corr(method='spearman')[0][1:]

#Plotting output correlation
plt.scatter(range(len(correlation_sparse)), np.sort(correlation_sparse))